In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [2]:
# use this code block to set column and row viewing size/width
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 40)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

pd.options.display.float_format = '{:.4f}'.format

In [3]:
# read the metadata csv
pbsummary_df = pd.read_csv('../metadata.csv', delimiter=';')
pbsummary_df = pbsummary_df.drop_duplicates()
pbsummary_df

pbsummary_aarau = pd.read_csv('../metadata_aarau.csv', delimiter=';')
pbsummary_df = pd.concat([pbsummary_df, pbsummary_aarau], ignore_index=True)

pbsummary_df['subunit'].fillna(value='all', inplace=True)

In [4]:
# Get election id and vote type, so we know better to segragate approval and score votings
pbsummary_with_vote_type = pbsummary_df[['election_id', 'vote_type']]
print(pbsummary_with_vote_type.head())
print(pbsummary_with_vote_type['vote_type'].value_counts())

   election_id   vote_type
0            1  cumulative
1            2  cumulative
2            3  cumulative
3            4  cumulative
4            5  cumulative
vote_type
approval      613
cumulative    104
ordinal        95
Name: count, dtype: int64


In [5]:
# reading the projects CSV and loading to dataframe
pbprojects_df = pd.read_csv('../projects.csv', delimiter=';')
pbprojects_df.drop_duplicates(inplace=True)
print(pbprojects_df.shape)

pbprojects_aarau = pd.read_csv('../projects_aarau.csv', delimiter=';')
pbprojects_df = pd.concat([pbprojects_df, pbprojects_aarau], ignore_index=True)

# merge the column vote_type into pbprojects_df
pbprojects_df = pd.merge(pbprojects_df, pbsummary_with_vote_type, on='election_id', how='inner')
print(pbprojects_df.shape)

pbprojects_df.head()

(19077, 21)
(19110, 22)


,election_id,country,unit,subunit,instance,project_id,cost,project_voters_count,votes,score,category,name,englishName,target,total_budget,budget_percent,is_mes_winner,is_greedy_winner,is_phragmen_winner,latitude,longitude,vote_type
0,1,France,Toulouse,ARENES,2019,1,7000,1494,215,NaN,NaN,Compostons ensemble !,NaN,NaN,1000000.0000,0.7000,True,True,True,NaN,NaN,cumulative
1,1,France,Toulouse,BELLEFONTAINE MILAN,2019,2,35000,1494,26,NaN,NaN,Panneau d'affichage électronique extérieur,NaN,NaN,1000000.0000,3.5000,False,False,False,NaN,NaN,cumulative
2,1,France,Toulouse,ARENES,2019,3,50000,1494,92,NaN,NaN,Eclairage public d’un chemin piéton-vélo rue Ella Maillart,NaN,NaN,1000000.0000,5.0000,True,False,True,NaN,NaN,cumulative
3,1,France,Toulouse,PRADETTES,2019,4,390000,1494,471,NaN,NaN,"Tous à la Ramée à vélo ! A pied, en trottinette et rollers !",NaN,NaN,1000000.0000,39.0000,False,True,False,NaN,NaN,cumulative
4,1,France,Toulouse,NEGRENEYS,2019,5,168000,1494,205,NaN,NaN,Le parc des Anges vu par ses usagers,NaN,NaN,1000000.0000,16.8000,True,True,True,NaN,NaN,cumulative


In [6]:

invalid_projects = pbprojects_df[pbprojects_df['cost'] == 0][['project_id','election_id']]
print(invalid_projects)

# Excluding that single project id by checking with particular election id and project id
valid_pbprojects_df = pbprojects_df[~(pbprojects_df['project_id'].isin(invalid_projects['project_id']) & (pbprojects_df['election_id'].isin(invalid_projects['election_id'])))]
print(pbprojects_df.shape)
print(valid_pbprojects_df.shape)

    project_id  election_id
893          5           72
(19110, 22)
(19109, 22)


In [7]:
print("Subunit empty projects count: ", valid_pbprojects_df['subunit'].isna().sum())

empty_subunit_projects_df = valid_pbprojects_df[valid_pbprojects_df['subunit'].isna()][['election_id','project_id']]
print(empty_subunit_projects_df.shape)
print(empty_subunit_projects_df['election_id'].nunique())

# There are citywide elections (unit-level); i.e. subunit is na, for such records fill na columns with values for subunit as all
valid_pbprojects_df['subunit'].fillna(value='all', inplace=True)

# Cross checking for projects where by we have filled with subunit equalling the value 'all'
print(valid_pbprojects_df[valid_pbprojects_df['subunit'] == 'all'].shape)
print("Valid PB projects are: ", valid_pbprojects_df.shape)

Subunit empty projects count:  1928
(1928, 2)
25
(1928, 22)
Valid PB projects are:  (19109, 22)


C:\Users\scsmah\AppData\Local\Temp\ipykernel_13268\660040707.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_pbprojects_df['subunit'].fillna(value='all', inplace=True)


In [8]:
# setting up vote_percent column
print(valid_pbprojects_df.shape)
valid_pb_projects_total_selections = valid_pbprojects_df.groupby(['election_id'])['votes'].sum().reset_index()
print(valid_pb_projects_total_selections.shape)
valid_pb_projects_total_selections.rename(columns={'votes': 'total_votes_selection'}, inplace=True)
valid_pbprojects_df = valid_pbprojects_df.merge(valid_pb_projects_total_selections, on='election_id', how='inner')
valid_pbprojects_df['vote_percent'] = round((valid_pbprojects_df['votes'] / valid_pbprojects_df['total_votes_selection'] * 100),3)
print(valid_pbprojects_df.shape)

(19109, 22)
(812, 2)
(19109, 24)


In [9]:
# Getting the total budget usage for MES winning projects of each election_id (grouped) and adding a new column to denote that value
valid_pb_projects_mes_budget_usage = valid_pbprojects_df[valid_pbprojects_df['is_mes_winner'] == True].groupby(['election_id'])['cost'].sum().reset_index()
valid_pb_projects_mes_budget_usage.rename(columns={'cost': 'mes_total_budget_usage'}, inplace=True)
valid_pbprojects_df = valid_pbprojects_df.merge(valid_pb_projects_mes_budget_usage, on='election_id', how='inner')
valid_pbprojects_df['mes_budget_usage_percent'] = round((valid_pbprojects_df['mes_total_budget_usage'] / valid_pbprojects_df['total_budget'] * 100),3)

# Getting the total budget usage for utilitarian greedy winning projects of each election_id (grouped) and adding a new column to denote that value
valid_pb_projects_greedy_budget_usage = valid_pbprojects_df[valid_pbprojects_df['is_greedy_winner']].groupby(['election_id'])['cost'].sum().reset_index()
valid_pb_projects_greedy_budget_usage.rename(columns={'cost': 'greedy_total_budget_usage'}, inplace=True)
valid_pbprojects_df = valid_pbprojects_df.merge(valid_pb_projects_greedy_budget_usage, on='election_id', how='inner')
valid_pbprojects_df['greedy_budget_usage_percent'] = round((valid_pbprojects_df['greedy_total_budget_usage'] / valid_pbprojects_df['total_budget'] * 100),3)

In [10]:
valid_pbprojects_df['category'].value_counts()

# Checking to see if there are empty values for category in the entire project dataset
na_category_count = valid_pbprojects_df['category'].isna().sum()
print("Empty category values for PB projects are: ", na_category_count)

# Fill such empty values of category with the label uncategorized, so that it can aid in further data preprocessing
valid_pbprojects_df['category'].fillna('uncategorized', inplace=True)


print("Emtpy category values after filling as uncategorized: ", valid_pbprojects_df['category'].isna().sum())
print("`uncategorized` category count for valid pb projects: ", valid_pbprojects_df[valid_pbprojects_df['category'] == 'uncategorized'].shape[0])

Empty category values for PB projects are:  8182
Emtpy category values after filling as uncategorized:  0
`uncategorized` category count for valid pb projects:  8182


## Create additional columns for each category label

In [11]:
# Split the Categories column and create a set of unique categories
categories_set = set(category.strip() for categories in valid_pbprojects_df['category'] for category in categories.split(','))

# Create new columns with default value 0
for category in categories_set:
    valid_pbprojects_df[f'category_{category}'] = 0

# Iterate through rows and update the new columns
for index, row in valid_pbprojects_df.iterrows():
    categories = row['category'].split(',')
    for category in categories:
        valid_pbprojects_df.at[index, f'category_{category.strip()}'] = 1
    
    if(len(categories) == 1 and (categories[0] == 'uncategorized')):
        valid_pbprojects_df.at[index, 'category_labels_count'] = 0
    else:
        valid_pbprojects_df.at[index, 'category_labels_count'] = int(len(categories))

valid_pbprojects_df.tail()

,election_id,country,unit,subunit,instance,project_id,cost,project_voters_count,votes,score,category,name,englishName,target,total_budget,budget_percent,is_mes_winner,is_greedy_winner,is_phragmen_winner,latitude,longitude,vote_type,total_votes_selection,vote_percent,mes_total_budget_usage,mes_budget_usage_percent,greedy_total_budget_usage,greedy_budget_usage_percent,category_welfare,category_urban greenery,category_sport,category_health,category_public space,category_culture,category_uncategorized,category_environmental protection,category_education,category_public transit and roads,category_labels_count
19104,1000,Switzerland,Aarau,Aarau,2023,18,5000,1703,88,146.0000,"health,education,welfare",Processing birth,NaN,"adults,families with children",50000.0000,10.0000,False,False,False,NaN,NaN,cumulative,8262,1.0650,49400,98.8000,49600,99.2000,1,0,0,1,0,0,0,0,1,0,3.0000
19105,1000,Switzerland,Aarau,Aarau,2023,17,3500,1703,67,102.0000,welfare,Senior Meeting,NaN,elderly,50000.0000,7.0000,False,False,False,NaN,NaN,cumulative,8262,0.8110,49400,98.8000,49600,99.2000,1,0,0,0,0,0,0,0,0,0,1.0000
19106,1000,Switzerland,Aarau,Aarau,2023,27,4000,1703,69,97.0000,"public space,public transit and roads",New edition of Telli-Map,NaN,"children,families with children,elderly",50000.0000,8.0000,False,False,False,NaN,NaN,cumulative,8262,0.8350,49400,98.8000,49600,99.2000,0,0,0,0,1,0,0,0,0,1,2.0000
19107,1000,Switzerland,Aarau,Aarau,2023,19,500,1703,56,85.0000,"welfare,education,health",Ways of Remembering,NaN,"youth,adults,elderly",50000.0000,1.0000,True,False,True,NaN,NaN,cumulative,8262,0.6780,49400,98.8000,49600,99.2000,1,0,0,1,0,0,0,0,1,0,3.0000
19108,1000,Switzerland,Aarau,Aarau,2023,6,4000,1703,44,85.0000,welfare,New Year 2024 Greetings,NaN,"youth,adults,families with children,elderly",50000.0000,8.0000,False,False,False,NaN,NaN,cumulative,8262,0.5330,49400,98.8000,49600,99.2000,1,0,0,0,0,0,0,0,0,0,1.0000


## For each instance of PB normalize project costs with min-max normalization

In [12]:
instance_grouped_max_pbprojects_df = valid_pbprojects_df.groupby(['election_id'])['cost'].max().reset_index()
instance_grouped_max_pbprojects_df.rename(columns={'cost': 'election_project_max_cost'}, inplace=True)

instance_grouped_min_pbprojects_df = valid_pbprojects_df.groupby(['election_id'])['cost'].min().reset_index()
instance_grouped_min_pbprojects_df.rename(columns={'cost': 'election_project_min_cost'}, inplace=True)

instance_grouped_total_projects_cost_df = valid_pbprojects_df.groupby(['election_id'])['cost'].sum().reset_index()
instance_grouped_total_projects_cost_df.rename(columns={'cost': 'total_projects_cost'}, inplace=True)

print("Valid PB Projects Shape before merge: ", valid_pbprojects_df.shape)
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_max_pbprojects_df, on='election_id', how='inner')
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_min_pbprojects_df, on='election_id', how='inner')
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_total_projects_cost_df, on='election_id', how='inner')
print("After merge: ", valid_pbprojects_df.shape)

def applyMinMaxNormalization(row):
    if ((row['cost'] == row['election_project_max_cost']) & (row['cost'] == row['election_project_min_cost'])):
        return 0.5
    else:
        return ((row['cost'] - row['election_project_min_cost']) / (row['election_project_max_cost'] - row['election_project_min_cost']))

valid_pbprojects_df['normalized_cost'] = valid_pbprojects_df.apply(lambda row: applyMinMaxNormalization(row), axis=1)
valid_pbprojects_df

Valid PB Projects Shape before merge:  (19109, 39)
After merge:  (19109, 42)


,election_id,country,unit,subunit,instance,project_id,cost,project_voters_count,votes,score,category,name,englishName,target,total_budget,budget_percent,is_mes_winner,is_greedy_winner,is_phragmen_winner,latitude,longitude,vote_type,total_votes_selection,vote_percent,mes_total_budget_usage,mes_budget_usage_percent,greedy_total_budget_usage,greedy_budget_usage_percent,category_welfare,category_urban greenery,category_sport,category_health,category_public space,category_culture,category_uncategorized,category_environmental protection,category_education,category_public transit and roads,category_labels_count,election_project_max_cost,election_project_min_cost,total_projects_cost,normalized_cost
0,1,France,Toulouse,ARENES,2019,1,7000,1494,215,NaN,uncategorized,Compostons ensemble !,NaN,NaN,1000000.0000,0.7000,True,True,True,NaN,NaN,cumulative,4252,5.0560,997000,99.7000,1000000,100.0000,0,0,0,0,0,0,1,0,0,0,0.0000,390000,2000,2452000,0.0129
1,1,France,Toulouse,BELLEFONTAINE MILAN,2019,2,35000,1494,26,NaN,uncategorized,Panneau d'affichage électronique extérieur,NaN,NaN,1000000.0000,3.5000,False,False,False,NaN,NaN,cumulative,4252,0.6110,997000,99.7000,1000000,100.0000,0,0,0,0,0,0,1,0,0,0,0.0000,390000,2000,2452000,0.0851
2,1,France,Toulouse,ARENES,2019,3,50000,1494,92,NaN,uncategorized,Eclairage public d’un chemin piéton-vélo rue Ella Maillart,NaN,NaN,1000000.0000,5.0000,True,False,True,NaN,NaN,cumulative,4252,2.1640,997000,99.7000,1000000,100.0000,0,0,0,0,0,0,1,0,0,0,0.0000,390000,2000,2452000,0.1237
3,1,France,Toulouse,PRADETTES,2019,4,390000,1494,471,NaN,uncategorized,"Tous à la Ramée à vélo ! A pied, en trottinette et rollers !",NaN,NaN,1000000.0000,39.0000,False,True,False,NaN,NaN,cumulative,4252,11.0770,997000,99.7000,1000000,100.0000,0,0,0,0,0,0,1,0,0,0,0.0000,390000,2000,2452000,1.0000
4,1,France,Toulouse,NEGRENEYS,2019,5,168000,1494,205,NaN,uncategorized,Le parc des Anges vu par ses usagers,NaN,NaN,1000000.0000,16.8000,True,True,True,NaN,NaN,cumulative,4252,4.8210,997000,99.7000,1000000,100.0000,0,0,0,0,0,0,1,0,0,0,0.0000,390000,2000,2452000,0.4278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19104,1000,Switzerland,Aarau,Aarau,2023,18,5000,1703,88,146.0000,"health,education,welfare",Processing birth,NaN,"adults,families with children",50000.0000,10.0000,False,False,False,NaN,NaN,cumulative,8262,1.0650,49400,98.8000,49600,99.2000,1,0,0,1,0,0,0,0,1,0,3.0000,25000,500,247330,0.1837
19105,1000,Switzerland,Aarau,Aarau,2023,17,3500,1703,67,102.0000,welfare,Senior Meeting,NaN,elderly,50000.0000,7.0000,False,False,False,NaN,NaN,cumulative,8262,0.8110,49400,98.8000,49600,99.2000,1,0,0,0,0,0,0,0,0,0,1.0000,25000,500,247330,0.1224
19106,1000,Switzerland,Aarau,Aarau,2023,27,4000,1703,69,97.0000,"public space,public transit and roads",New edition of Telli-Map,NaN,"children,families with children,elderly",50000.0000,8.0000,False,False,False,NaN,NaN,cumulative,8262,0.8350,49400,98.8000,49600,99.2000,0,0,0,0,1,0,0,0,0,1,2.0000,25000,500,247330,0.1429
19107,1000,Switzerland,Aarau,Aarau,2023,19,500,1703,56,85.0000,"welfare,education,health",Ways of Remembering,NaN,"youth,adults,elderly",50000.0000,1.0000,True,False,True,NaN,NaN,cumulative,8262,0.6780,49400,98.8000,49600,99.2000,1,0,0,1,0,0,0,0,1,0,3.0000,25000,500,247330,0.0000


In [13]:
valid_pbprojects_df.columns

Index(['election_id', 'country', 'unit', 'subunit', 'instance', 'project_id',
       'cost', 'project_voters_count', 'votes', 'score', 'category', 'name',
       'englishName', 'target', 'total_budget', 'budget_percent',
       'is_mes_winner', 'is_greedy_winner', 'is_phragmen_winner', 'latitude',
       'longitude', 'vote_type', 'total_votes_selection', 'vote_percent',
       'mes_total_budget_usage', 'mes_budget_usage_percent',
       'greedy_total_budget_usage', 'greedy_budget_usage_percent',
       'category_welfare', 'category_urban greenery', 'category_sport',
       'category_health', 'category_public space', 'category_culture',
       'category_uncategorized', 'category_environmental protection',
       'category_education', 'category_public transit and roads',
       'category_labels_count', 'election_project_max_cost',
       'election_project_min_cost', 'total_projects_cost', 'normalized_cost'],
      dtype='object')

<font color="orange">
So, applying the above filter with loss_metric_two and loss_metric_three equals 1, we notice that these cases are those PB cases, where PB projects values are skewed, i.e. one or many of the popular projects utilize 100% of the budget, while there are relatively smaller budget projects that are selected by MES with lower popularity, thus resulting in less budget usage. These PB instances are those with smaller number of projects too. So using inference, a city level project was not seen, using this filter too
</font>

## Create a replica of the valid_pbprojects_df and find the distribution % of PB winners across different categories using MES against Greedy

In [14]:
categorization_df = valid_pbprojects_df[valid_pbprojects_df['category'] != 'uncategorized']
categorization_df.rename(columns={
       'category_education': 'education', 
       'category_public transit and roads': 'public_transit_and_roads',
       'category_health': 'health', 
       'category_welfare': 'welfare',
       'category_uncategorized':'uncategorized',
       'category_public space': 'public_space',
       'category_urban greenery': 'urban_greenery',
       'category_culture': 'culture', 
       'category_sport': 'sport', 
       'category_environmental protection': 'env_protection',
       'category_labels_count': 'total_tags' 
       }, inplace=True)


C:\Users\scsmah\AppData\Local\Temp\ipykernel_13268\823576670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorization_df.rename(columns={


In [15]:
# some globals to be used for the code snippets below
categories_set = ['education', 'public_transit_and_roads', 'health', 'welfare', 'public_space', 'urban_greenery', 'culture', 'sport', 'env_protection']

category_title_map = {
    'education': 'Education',
    'public_transit_and_roads': 'Public Transit',
    'health': 'Health',
    'welfare': 'Welfare',
    'public_space': 'Public Space',
    'urban_greenery': 'Urban Greenery', 
    'culture': 'Culture', 
    'sport': 'Sport',
    'env_protection': 'Env. Protection'
}

oneD_to_twoD_map = {
    0: [0,0],
    1: [0,1],
    2: [0,2],
    3: [1,0],
    4: [1,1],
    5: [1,2],
    6: [2,0],
    7: [2,1],
    8: [2,2]
}

In [16]:
# total cost of projects labeled under each category at each election
print("Shape of categorization_df : ", categorization_df.shape)

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[categorization_df[category] == 1].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'{category}_total_cost'].fillna(0, inplace=True)

Shape of categorization_df :  (10927, 43)


In [17]:
# Add 4 columns that denote; MES Winners Projects Count, MES Winners Project Costs, Greedy Winners Projects Count and Greedy Winners Project Cost; (overall)
election_grouped_mes_winners_total_cost = categorization_df[(categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_mes_winners_total_cost.rename(columns={'cost': 'mes_winners_total_cost'}, inplace=True)

election_grouped_greedy_winners_total_cost = categorization_df[(categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_greedy_winners_total_cost.rename(columns={'cost': 'greedy_winners_total_cost'}, inplace=True)

election_grouped_mes_winners_total_project_count = categorization_df[(categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_mes_winners_total_project_count.rename(columns={'project_id': 'mes_winners_projects_count'}, inplace=True)

election_grouped_greedy_winners_total_project_count = categorization_df[(categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_greedy_winners_total_project_count.rename(columns={'project_id': 'greedy_winners_projects_count'}, inplace=True)

# Merge these dataset with categorization_df
categorization_df = categorization_df.merge(election_grouped_mes_winners_total_cost, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_greedy_winners_total_cost, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_mes_winners_total_project_count, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_greedy_winners_total_project_count, on='election_id', how='inner')

In [18]:
# Add another 4 columns that denote; only MES Winners Projects Count, only MES Winners Project Costs, only Greedy Winners Projects Count and only Greedy Winners Project Cost; (overall)
election_grouped_only_mes_winners_total_cost = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_only_mes_winners_total_cost.rename(columns={'cost': 'only_mes_winners_total_cost'}, inplace=True)

election_grouped_only_greedy_winners_total_cost = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_only_greedy_winners_total_cost.rename(columns={'cost': 'only_greedy_winners_total_cost'}, inplace=True)

election_grouped_only_mes_winners_total_project_count = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_only_mes_winners_total_project_count.rename(columns={'project_id': 'only_mes_winners_projects_count'}, inplace=True)

election_grouped_only_greedy_winners_total_project_count = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_only_greedy_winners_total_project_count.rename(columns={'project_id': 'only_greedy_winners_projects_count'}, inplace=True)

# Merge these dataset with categorization_df; for only, we will need to use left join, because there can be cases where
# both outcomes are the same, in such cases only values don't exist; so left join is needed
categorization_df = categorization_df.merge(election_grouped_only_mes_winners_total_cost, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_greedy_winners_total_cost, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_mes_winners_total_project_count, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_greedy_winners_total_project_count, on='election_id', how='left')

In [19]:
# Merge categorization_df with pbsummary to get total projects in all elections
pbsummary_num_votes = pbsummary_df[['election_id', 'num_projects']]
categorization_df = categorization_df.merge(pbsummary_num_votes, on='election_id', how='inner')
categorization_df

,election_id,country,unit,subunit,instance,project_id,cost,project_voters_count,votes,score,category,name,englishName,target,total_budget,budget_percent,is_mes_winner,is_greedy_winner,is_phragmen_winner,latitude,longitude,vote_type,total_votes_selection,vote_percent,mes_total_budget_usage,mes_budget_usage_percent,greedy_total_budget_usage,greedy_budget_usage_percent,welfare,urban_greenery,sport,health,public_space,culture,uncategorized,env_protection,education,public_transit_and_roads,total_tags,election_project_max_cost,election_project_min_cost,total_projects_cost,normalized_cost,education_total_cost,public_transit_and_roads_total_cost,health_total_cost,welfare_total_cost,public_space_total_cost,urban_greenery_total_cost,culture_total_cost,sport_total_cost,env_protection_total_cost,mes_winners_total_cost,greedy_winners_total_cost,mes_winners_projects_count,greedy_winners_projects_count,only_mes_winners_total_cost,only_greedy_winners_total_cost,only_mes_winners_projects_count,only_greedy_winners_projects_count,num_projects
0,332,Poland,Warszawa,Aleksandrów,2017,261,80000,422,172,NaN,"public transit and roads,public space",REMONT NAWIERZCHNI PŁYTOWEJ ULICY ZAGÓRZAŃSKIEJ OD NR 50D DO KOŃCA W STRONĘ LASU,NaN,"adults,families with children",110411.0000,72.4570,False,True,False,NaN,NaN,approval,615,27.9670,107950,97.7710,110000,99.6280,0,0,0,0,1,0,0,0,0,1,2.0000,99267,9200,287217,0.7861,39200.0000,248017.0000,0.0000,0.0000,110000.0000,0.0000,30000.0000,9200.0000,0.0000,107950,110000,3,2,77950.0000,80000.0000,2.0000,1.0000,5
1,332,Poland,Warszawa,Aleksandrów,2017,1112,99267,422,140,NaN,public transit and roads,UTWARDZENIE NAWIERZCHNI NA ULICY JASNOTY,NaN,"adults,seniors,families with children",110411.0000,89.9070,False,False,False,NaN,NaN,approval,615,22.7640,107950,97.7710,110000,99.6280,0,0,0,0,0,0,0,0,0,1,1.0000,99267,9200,287217,1.0000,39200.0000,248017.0000,0.0000,0.0000,110000.0000,0.0000,30000.0000,9200.0000,0.0000,107950,110000,3,2,77950.0000,80000.0000,2.0000,1.0000,5
2,332,Poland,Warszawa,Aleksandrów,2017,720,30000,422,118,NaN,"public space,education,culture",Artystyczna pracownia letnia,NaN,NaN,110411.0000,27.1710,True,True,True,NaN,NaN,approval,615,19.1870,107950,97.7710,110000,99.6280,0,0,0,0,1,1,0,0,1,0,3.0000,99267,9200,287217,0.2309,39200.0000,248017.0000,0.0000,0.0000,110000.0000,0.0000,30000.0000,9200.0000,0.0000,107950,110000,3,2,77950.0000,80000.0000,2.0000,1.0000,5
3,332,Poland,Warszawa,Aleksandrów,2017,1206,9200,422,99,NaN,"education,sport",Bezpieczna Kobieta - Kurs samoobrony,NaN,"adults,seniors",110411.0000,8.3330,True,False,True,NaN,NaN,approval,615,16.0980,107950,97.7710,110000,99.6280,0,0,1,0,0,0,0,0,1,0,2.0000,99267,9200,287217,0.0000,39200.0000,248017.0000,0.0000,0.0000,110000.0000,0.0000,30000.0000,9200.0000,0.0000,107950,110000,3,2,77950.0000,80000.0000,2.0000,1.0000,5
4,332,Poland,Warszawa,Aleksandrów,2017,2592,68750,422,86,NaN,public transit and roads,Budowa chodnika w ul. Optymistów,NaN,"children,youth,adults",110411.0000,62.2670,True,False,True,NaN,NaN,approval,615,13.9840,107950,97.7710,110000,99.6280,0,0,0,0,0,0,0,0,0,1,1.0000,99267,9200,287217,0.6612,39200.0000,248017.0000,0.0000,0.0000,110000.0000,0.0000,30000.0000,9200.0000,0.0000,107950,110000,3,2,77950.0000,80000.0000,2.0000,1.0000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10922,1000,Switzerland,Aarau,Aarau,2023,18,5000,1703,88,146.0000,"health,education,welfare",Processing birth,NaN,"adults,families with children",50000.0000,10.0000,False,False,False,NaN,NaN,cumulative,8262,1.0650,49400,98.8000,49600,99.2000,1,0,0,1,0,0,0,0,1,0,3.0000,25000,500,247330,0.1837,57800.0000,8000.0000,23300.0000,91900.0000,155730.0000,76400.0000,86700.0000,33500.0000,59600.0000,49400,49600,17,7,29800.0000,30000.0000,12.0000,2.0000,33
10923,1000,Switzerland,A

In [20]:
# Adding total cost of each category in each election for greedy winners
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    greedy_winners_category_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
    greedy_winners_category_grouped_df.rename(columns={'cost': f'greedy_winners_{category}_total_cost'}, inplace=True)
    
    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(greedy_winners_category_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'greedy_winners_{category}_total_cost'].fillna(0, inplace=True)

# Adding total cost of each category in each election for MES winners
for category in categories_set:
    # temporarily grouped df 
    mes_winners_category_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
    mes_winners_category_grouped_df.rename(columns={'cost': f'mes_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(mes_winners_category_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'mes_winners_{category}_total_cost'].fillna(0, inplace=True)


In [21]:
# Project counts in each category; just overall, not winners in any sense
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[categorization_df[category] == 1].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'{category}_projects_count'].fillna(0, inplace=True)

##############################################################################################
# Project counts in each category that are Greedy winners
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'greedy_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'greedy_winners_{category}_projects_count'].fillna(0, inplace=True)

##############################################################################################
# Project counts in each category that are MES Winners
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'mes_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'mes_winners_{category}_projects_count'].fillna(0, inplace=True)

##############################################################################################
# Project counts in each category that are Greedy winners only
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'only_greedy_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_greedy_winners_{category}_projects_count'].fillna(0, inplace=True)

##############################################################################################
# Project counts in each category that are MES Winners only
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'only_mes_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_mes_winners_{category}_projects_count'].fillna(0, inplace=True)

In [22]:
# Project counts in each category that are Greedy winners only
categories_set = ['education', 'public_transit_and_roads', 'health', 'welfare', 'public_space', 'urban_greenery', 'culture', 'sport', 'env_protection']

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'only_greedy_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_greedy_winners_{category}_total_cost'].fillna(0, inplace=True)


# Project counts in each category that are MES Winners only
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'only_mes_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_mes_winners_{category}_total_cost'].fillna(0, inplace=True)

In [23]:
print(categorization_df['vote_type'].value_counts()) # the 33 cumulative ballot types are for Aarau City Idea

categorization_df['total_votes'] = np.where(categorization_df['vote_type'] == 'approval', categorization_df['votes'], categorization_df['score'])

categorization_df.drop(['votes', 'score'], axis=1, inplace=True)


categorization_df = categorization_df.merge(pbsummary_df[['election_id', 'num_votes']], on='election_id', how='inner')
categorization_df.rename(columns={'num_votes': 'num_voters'}, inplace=True)

vote_type
approval      10894
cumulative       33
Name: count, dtype: int64


In [24]:
## Metrics Required for Popularity Proportionality
### total sum of votes received by all projects across an indvidiual instance
num_votes_all_projects_per_instance = categorization_df.groupby(['election_id'])['total_votes'].sum().reset_index()
num_votes_all_projects_per_instance.rename(columns={'total_votes': 'all_projects_total_votes_received_in_instance'}, inplace=True)

print(num_votes_all_projects_per_instance[num_votes_all_projects_per_instance['election_id'] == 332]['all_projects_total_votes_received_in_instance']) # should be 615; for cross-checking

categorization_df = categorization_df.merge(num_votes_all_projects_per_instance, on='election_id', how='inner')

categories_set = ['education', 'health', 'culture', 'welfare', 'sport', 'public_space', 'public_transit_and_roads', 'urban_greenery', 'env_protection']

## for individual impact area; total votes received
for category in categories_set:
    category_all_projects_votes_per_instance = categorization_df[categorization_df[f'{category}'] == 1].groupby(['election_id'])['total_votes'].sum().reset_index()
    category_all_projects_votes_per_instance.rename(columns={'total_votes': f'{category}_total_votes_received_in_instance'}, inplace=True)

    categorization_df = categorization_df.merge(category_all_projects_votes_per_instance, on='election_id', how='left')
    
for category in categories_set:
    categorization_df[f'{category}_total_votes_received_in_instance'].fillna(0, inplace=True)

## for es winning projects; total votes received
es_num_votes_all_projects_per_instance = categorization_df[categorization_df['is_mes_winner'] == True].groupby(['election_id'])['total_votes'].sum().reset_index()
es_num_votes_all_projects_per_instance.rename(columns={'total_votes': 'es_all_projects_total_votes_received_in_instance'}, inplace=True)

# print(es_num_votes_all_projects_per_instance[es_num_votes_all_projects_per_instance['election_id' == 332]]) # should be about 303
categorization_df = categorization_df.merge(es_num_votes_all_projects_per_instance, on='election_id', how='inner')

## for ug winning projects; total votes received
ug_num_votes_all_projects_per_instance = categorization_df[categorization_df['is_greedy_winner'] == True].groupby(['election_id'])['total_votes'].sum().reset_index()
ug_num_votes_all_projects_per_instance.rename(columns={'total_votes': 'ug_all_projects_total_votes_received_in_instance'}, inplace=True)

# print(ug_num_votes_all_projects_per_instance[ug_num_votes_all_projects_per_instance['election_id'] == 332]['ug_all_projects_total_votes_received_in_instance']) # should be about 290
categorization_df = categorization_df.merge(ug_num_votes_all_projects_per_instance, on='election_id', how='inner')

### for individual impact areas
for category in categories_set:
    # for es winning in individual impact area
    category_es_num_votes_projects_per_instance = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df[f'{category}'] == 1)].groupby(['election_id'])['total_votes'].sum().reset_index()
    category_es_num_votes_projects_per_instance.rename(columns={'total_votes': f'{category}_es_total_votes_received_in_instance'}, inplace=True)
    categorization_df = categorization_df.merge(category_es_num_votes_projects_per_instance, on='election_id', how='left')
    
    # for ug winning in individual impact area
    category_ug_num_votes_projects_per_instance = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df[f'{category}'] == 1)].groupby(['election_id'])['total_votes'].sum().reset_index()
    category_ug_num_votes_projects_per_instance.rename(columns={'total_votes': f'{category}_ug_total_votes_received_in_instance'}, inplace=True)
    categorization_df = categorization_df.merge(category_ug_num_votes_projects_per_instance, on='election_id', how='left')
    
# applying fillna oepration in a separate loop; because it doesn't work in the above loop when merged
for category in categories_set:
    categorization_df[f'{category}_es_total_votes_received_in_instance'].fillna(0, inplace=True)
    categorization_df[f'{category}_ug_total_votes_received_in_instance'].fillna(0, inplace=True)


0   615.0000
Name: all_projects_total_votes_received_in_instance, dtype: float64


##  Impact Metrics

In [25]:
# metrics calculation required for relative winners
print("Current shape of categorization df is: ", categorization_df.shape)

for category in categories_set:
    # cost share
    categorization_df[f'ug_budget_share_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_total_cost'] / categorization_df['greedy_winners_total_cost']
    categorization_df[f'es_budget_share_{category}'] = 1 * categorization_df[f'mes_winners_{category}_total_cost'] / categorization_df['mes_winners_total_cost']
    categorization_df[f'diff_budget_share_{category}'] = categorization_df[f'ug_budget_share_{category}'] - categorization_df[f'es_budget_share_{category}']

    # popularity share
    categorization_df[f'ug_popularity_share_{category}'] = 1 * categorization_df[f'{category}_ug_total_votes_received_in_instance'] / categorization_df['ug_all_projects_total_votes_received_in_instance']
    categorization_df[f'es_popularity_share_{category}'] = 1 * categorization_df[f'{category}_es_total_votes_received_in_instance'] / categorization_df['es_all_projects_total_votes_received_in_instance']
    categorization_df[f'diff_popularity_share_{category}'] = categorization_df[f'ug_popularity_share_{category}'] - categorization_df[f'es_popularity_share_{category}']

    # projects share
    categorization_df[f'ug_winning_rate_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_projects_count'] / categorization_df['greedy_winners_projects_count']
    categorization_df[f'es_winning_rate_{category}'] = 1 * categorization_df[f'mes_winners_{category}_projects_count'] / categorization_df['mes_winners_projects_count']
    categorization_df[f'diff_winning_rate_{category}'] = categorization_df[f'ug_winning_rate_{category}'] - categorization_df[f'es_winning_rate_{category}']

    # cost representation
    categorization_df[f'ug_cost_representation_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'ug_cost_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'es_cost_representation_{category}'] = 1 * categorization_df[f'mes_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'es_cost_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'diff_cost_representation_{category}'] = categorization_df[f'ug_cost_representation_{category}'] - categorization_df[f'es_cost_representation_{category}']
    
    # project representation
    categorization_df[f'ug_project_representation_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'ug_project_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'es_project_representation_{category}'] = 1 * categorization_df[f'mes_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'es_project_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'diff_project_representation_{category}'] = categorization_df[f'ug_project_representation_{category}'] - categorization_df[f'es_project_representation_{category}']

    # popularity representation
    categorization_df[f'ug_popularity_representation_{category}'] = 1 * categorization_df[f'{category}_ug_total_votes_received_in_instance'] / categorization_df[f'{category}_total_votes_received_in_instance']
    categorization_df[f'ug_popularity_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'es_popularity_representation_{category}'] = 1 * categorization_df[f'{category}_es_total_votes_received_in_instance'] / categorization_df[f'{category}_total_votes_received_in_instance']
    categorization_df[f'es_popularity_representation_{category}'].fillna(0, inplace=True)
    categorization_df[f'diff_popularity_representation_{category}'] = categorization_df[f'ug_popularity_representation_{category}'] - categorization_df[f'es_popularity_representation_{category}']

    # cost proportionality
    categorization_df[f'ug_cost_proportionality_{category}'] = (categorization_df[f'greedy_winners_{category}_total_cost'] / categorization_df['greedy_winners_total_cost']) / (categorization_df[f'{category}_total_cost'] / categorization_df['total_projects_cost'])
    categorization_df[f'es_cost_proportionality_{category}'] = (categorization_df[f'mes_winners_{category}_total_cost'] / categorization_df['mes_winners_total_cost']) / (categorization_df[f'{category}_total_cost'] / categorization_df['total_projects_cost'])
    categorization_df[f'diff_cost_proportionality_{category}'] = categorization_df[f'ug_cost_proportionality_{category}'] - categorization_df[f'es_cost_proportionality_{category}']

    # project proportionality
    categorization_df[f'ug_project_proportionality_{category}'] = (categorization_df[f'greedy_winners_{category}_projects_count'] / categorization_df['greedy_winners_projects_count']) / (categorization_df[f'{category}_projects_count'] / categorization_df['num_projects'])
    categorization_df[f'es_project_proportionality_{category}'] = (categorization_df[f'mes_winners_{category}_projects_count'] / categorization_df['mes_winners_projects_count']) / (categorization_df[f'{category}_projects_count'] / categorization_df['num_projects'])
    categorization_df[f'diff_project_proportionality_{category}'] = categorization_df[f'ug_project_proportionality_{category}'] - categorization_df[f'es_project_proportionality_{category}']

    # popularity proportionality
    categorization_df[f'ug_popularity_proportionality_{category}'] = 1.00 * (categorization_df[f'{category}_ug_total_votes_received_in_instance'] / categorization_df['ug_all_projects_total_votes_received_in_instance']) / (categorization_df[f'{category}_total_votes_received_in_instance'] / categorization_df['all_projects_total_votes_received_in_instance'])
    categorization_df[f'es_popularity_proportionality_{category}'] = 1.00 * (categorization_df[f'{category}_es_total_votes_received_in_instance'] / categorization_df['es_all_projects_total_votes_received_in_instance']) / (categorization_df[f'{category}_total_votes_received_in_instance'] / categorization_df['all_projects_total_votes_received_in_instance'])


Current shape of categorization df is:  (10927, 172)


C:\Users\scsmah\AppData\Local\Temp\ipykernel_13268\2230232425.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  categorization_df[f'diff_popularity_representation_{category}'] = categorization_df[f'ug_popularity_representation_{category}'] - categorization_df[f'es_popularity_representation_{category}']
C:\Users\scsmah\AppData\Local\Temp\ipykernel_13268\2230232425.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  categorization_df[f'ug_cost_proportionality_{category}'] = (categorization_df[f'greedy_winners_{category}_total_

In [26]:
cols_set = ['election_id']

for category in categories_set:
    cols_set.append(f'ug_budget_share_{category}')
    cols_set.append(f'es_budget_share_{category}')
    cols_set.append(f'ug_popularity_share_{category}')
    cols_set.append(f'es_popularity_share_{category}')
    cols_set.append(f'ug_winning_rate_{category}')
    cols_set.append(f'es_winning_rate_{category}')
    cols_set.append(f'ug_cost_representation_{category}')
    cols_set.append(f'es_cost_representation_{category}')
    cols_set.append(f'ug_project_representation_{category}')
    cols_set.append(f'es_project_representation_{category}')
    cols_set.append(f'ug_popularity_representation_{category}')
    cols_set.append(f'es_popularity_representation_{category}')
    cols_set.append(f'ug_cost_proportionality_{category}')
    cols_set.append(f'es_cost_proportionality_{category}')
    cols_set.append(f'ug_project_proportionality_{category}')
    cols_set.append(f'es_project_proportionality_{category}')
    cols_set.append(f'ug_popularity_proportionality_{category}')
    cols_set.append(f'es_popularity_proportionality_{category}')


election_data = categorization_df[cols_set]
print(election_data.shape)

election_data.drop_duplicates(inplace=True)
print(election_data.shape)
    

(10927, 163)
(346, 163)


C:\Users\scsmah\AppData\Local\Temp\ipykernel_13268\2887969129.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_data.drop_duplicates(inplace=True)


In [27]:
election_data.head()

,election_id,ug_budget_share_education,es_budget_share_education,ug_popularity_share_education,es_popularity_share_education,ug_winning_rate_education,es_winning_rate_education,ug_cost_representation_education,es_cost_representation_education,ug_project_representation_education,es_project_representation_education,ug_popularity_representation_education,es_popularity_representation_education,ug_cost_proportionality_education,es_cost_proportionality_education,ug_project_proportionality_education,es_project_proportionality_education,ug_popularity_proportionality_education,es_popularity_proportionality_education,ug_budget_share_health,es_budget_share_health,ug_popularity_share_health,es_popularity_share_health,ug_winning_rate_health,es_winning_rate_health,ug_cost_representation_health,es_cost_representation_health,ug_project_representation_health,es_project_representation_health,ug_popularity_representation_health,es_popularity_representation_health,ug_cost_proportionality_health,es_cost_proportionality_health,ug_project_proportionality_health,es_project_proportionality_health,ug_popularity_proportionality_health,es_popularity_proportionality_health,ug_budget_share_culture,es_budget_share_culture,ug_popularity_share_culture,es_popularity_share_culture,ug_winning_rate_culture,es_winning_rate_culture,ug_cost_representation_culture,es_cost_representation_culture,ug_project_representation_culture,es_project_representation_culture,ug_popularity_representation_culture,es_popularity_representation_culture,ug_cost_proportionality_culture,es_cost_proportionality_culture,ug_project_proportionality_culture,es_project_proportionality_culture,ug_popularity_proportionality_culture,es_popularity_proportionality_culture,ug_budget_share_welfare,es_budget_share_welfare,ug_popularity_share_welfare,es_popularity_share_welfare,ug_winning_rate_welfare,es_winning_rate_welfare,ug_cost_representation_welfare,es_cost_representation_welfare,ug_project_representation_welfare,es_project_representation_welfare,ug_popularity_representation_welfare,es_popularity_representation_welfare,ug_cost_proportionality_welfare,es_cost_proportionality_welfare,ug_project_proportionality_welfare,es_project_proportionality_welfare,ug_popularity_proportionality_welfare,es_popularity_proportionality_welfare,ug_budget_share_sport,es_budget_share_sport,ug_popularity_share_sport,es_popularity_share_sport,ug_winning_rate_sport,es_winning_rate_sport,ug_cost_representation_sport,es_cost_representation_sport,ug_project_representation_sport,es_project_representation_sport,ug_popularity_representation_sport,es_popularity_representation_sport,ug_cost_proportionality_sport,es_cost_proportionality_sport,ug_project_proportionality_sport,es_project_proportionality_sport,ug_popularity_proportionality_sport,es_popularity_proportionality_sport,ug_budget_share_public_space,es_budget_share_public_space,ug_popularity_share_public_space,es_popularity_share_public_space,ug_winning_rate_public_space,es_winning_rate_public_space,ug_cost_representation_public_space,es_cost_representation_public_space,ug_project_representation_public_space,es_project_representation_public_space,ug_popularity_representation_public_space,es_popularity_representation_public_space,ug_cost_proportionality_public_space,es_cost_proportionality_public_space,ug_project_proportionality_public_space,es_project_proportionality_public_space,ug_popularity_proportionality_public_space,es_popularity_proportionality_public_space,ug_budget_share_public_transit_and_roads,es_budget_share_public_transit_and_roads,ug_popularity_share_public_transit_and_roads,es_popularity_share_public_transit_and_roads,ug_winning_rate_public_transit_and_roads,es_winning_rate_public_transit_and_roads,ug_cost_representation_public_transit_and_roads,es_cost_representation_public_transit_and_roads,ug_project_representation_public_transit_and_roads,es_project_representation_public_transit_and_roads,ug_popularity_representation_public_transit_and_roads,es_popularity

In [28]:
# paired t-tests
ttest_stats = pd.DataFrame(columns=['metric', 'category', 't-statistic', 'p-value'])

from scipy.stats import ttest_rel

print("\n----------------------Paired T-tests on Budget Share----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_budget_share_{category}'], election_data[f'es_budget_share_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Budget Share', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Winning Rate----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_winning_rate_{category}'], election_data[f'es_winning_rate_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Winning Rate', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Cost Representation----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_cost_representation_{category}'], election_data[f'es_cost_representation_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Cost Representation', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Project Representation----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_project_representation_{category}'], election_data[f'es_project_representation_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Project Representation', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Project Proportionality----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    temp_df = election_data[[f'ug_project_proportionality_{category}', f'es_project_proportionality_{category}']]
    temp_df_cleaned = temp_df.dropna()
    tstats, pvalue = ttest_rel(temp_df_cleaned[f'ug_project_proportionality_{category}'], temp_df_cleaned[f'es_project_proportionality_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Project Proportionality', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Cost Proportionality----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    temp_df = election_data[[f'ug_cost_proportionality_{category}', f'es_cost_proportionality_{category}']]
    temp_df_cleaned = temp_df.dropna()
    tstats, pvalue = ttest_rel(temp_df_cleaned[f'ug_cost_proportionality_{category}'], temp_df_cleaned[f'es_cost_proportionality_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Cost Proportionality', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n---------------------Paired T-tests on Popularity Proportionality----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    temp_df = election_data[[f'ug_popularity_proportionality_{category}', f'es_popularity_proportionality_{category}']]
    temp_df_cleaned = temp_df.dropna()
    tstats, pvalue = ttest_rel(temp_df_cleaned[f'ug_popularity_proportionality_{category}'], temp_df_cleaned[f'es_popularity_proportionality_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Popularity Proportionality', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Popularity Share----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_popularity_share_{category}'], election_data[f'es_popularity_share_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Popularity Share', 'category': category, 't-statistic': tstats, 'p-value': pvalue }

print("\n----------------------Paired T-tests on Popularity Representation----------------------")
for category in categories_set:
    category_title = category_title_map[category]
    tstats, pvalue = ttest_rel(election_data[f'ug_popularity_representation_{category}'], election_data[f'es_popularity_representation_{category}'])
    ttest_stats.loc[len(ttest_stats)] = {'metric': 'Popularity Representation', 'category': category, 't-statistic': tstats, 'p-value': pvalue }


----------------------Paired T-tests on Budget Share----------------------

----------------------Paired T-tests on Winning Rate----------------------

----------------------Paired T-tests on Cost Representation----------------------

----------------------Paired T-tests on Project Representation----------------------

----------------------Paired T-tests on Project Proportionality----------------------

----------------------Paired T-tests on Cost Proportionality----------------------

---------------------Paired T-tests on Popularity Proportionality----------------------

----------------------Paired T-tests on Popularity Share----------------------

----------------------Paired T-tests on Popularity Representation----------------------


In [29]:
ttest_stats.to_csv('./paired_t_test_results.csv', index=False)

In [30]:
ttest_stats.tail(20)

,metric,category,t-statistic,p-value
61,Popularity Proportionality,urban_greenery,7.7550,0.0000
62,Popularity Proportionality,env_protection,5.6037,0.0000
63,Popularity Share,education,-11.7589,0.0000
64,Popularity Share,health,0.7114,0.4773
65,Popularity Share,culture,-11.5966,0.0000
66,Popularity Share,welfare,-5.9495,0.0000
67,Popularity Share,sport,2.4293,0.0156
68,Popularity Share,public_space,7.0657,0.0000
69,Popularity Share,public_transit_and_roads,4.4529,0.0000
70,Popularity Share,urban_greenery,7.7528,0.0000
